# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость! — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 


<h1>Введение</h1>

Цель задачи определить параметры, влияющие на рыночную стоимость объектов недвижимости.

В нашем распоряжении есть файл с архивом объявлений о продажах недвижимости за некоторый период времени, содержащий следущие параметры.

Описание параметров.

airports_nearest — расстояние до ближайшего аэропорта в метрах (м)<br>
balcony — число балконов<br>
ceiling_height — высота потолков (м)<br>
cityCenters_nearest — расстояние до центра города (м)<br>
days_exposition — сколько дней было размещено объявление (от публикации до снятия)<br>
first_day_exposition — дата публикации<br>
floor — этаж<br>
floors_total — всего этажей в доме<br>
is_apartment — апартаменты (булев тип)<br>
kitchen_area — площадь кухни в квадратных метрах (м²)<br>
last_price — цена на момент снятия с публикации<br>
living_area — жилая площадь в квадратных метрах (м²)<br>
locality_name — название населённого пункта<br>
open_plan — свободная планировка (булев тип)<br>
parks_around3000 — число парков в радиусе 3 км<br>
parks_nearest — расстояние до ближайшего парка (м)<br>
ponds_around3000 — число водоёмов в радиусе 3 км<br>
ponds_nearest — расстояние до ближайшего водоёма (м)<br>
rooms — число комнат<br>
studio — квартира-студия (булев тип)<br>
total_area — общая площадь квартиры в квадратных метрах (м²)<br>
total_images — число фотографий квартиры в объявлении


В процессе работы предстоит изучить полученные данные, провести их предварительную обработку, добавить необходимые новые столбцы, после чего провести исследовательский анализ данных.


### Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:

data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
print(data.info())
data.hist(figsize=(15, 20));

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/real_estate_data.csv'

In [ ]:
data.head()

В полученных данных 22 столбца и 23699 строк с информацией. Гистограммы позволяют примерно понять как распределены числовые значения параметров. Некоторые гистограммы по умолчанию не слишком информативны, но по крайней мере позволяют увидеть разброс значений.

### Предобработка данных

Найдем явные задвоения, если такие имеются. Переименуем столбец cityCenters_nearest.

In [ ]:
data = data.rename(columns={'cityCenters_nearest':'city_centers_nearest'})
data.duplicated().sum()

Явных задвоений нет.

Пропущенные значения есть в столбцах ceiling_height, floors_total, living_area, is_apartment, kitchen_area, balcony, locality_name, airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition.  
Разберем каждый столбец с пропущенными значениями.  
  
**Высота потолка.**

In [ ]:
#ceiling_height - высота потолка
print(data['ceiling_height'].median())
data['ceiling_height'].isna().mean()

Доля пустых значений 38,79%. Медианное значение 2,65 выглядит правдоподобно. Заменим пустые значение медианным.

In [ ]:
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median())
data['ceiling_height'].isna().sum()

**Всего этажей в доме.**

In [ ]:
#floors_total - всего этажей
print(data['floors_total'].isna().sum()) # всего пустых  
data['floors_total'].isna().mean() #доля пустых


Незаполнено 86 раз, или всего 0,36%. Удалим незаполненные.

In [ ]:
data = data.dropna(subset=['floors_total'])
print(data['floors_total'].isna().sum())

<i><strike>Незаполнено 86 раз, или всего 0,36%. Не указано количество этажей в строке, но посмотрим, указан ли этаж для таких строк.</strike></i>

In [ ]:
#data[data['floors_total'].isna()]['floor'].isna().sum()

<i><strike>Пустых значений нет. Заменим пустые значения столбца floors_total на медианное значение столбца floors_total, но если значение floor больше, то возьмем его.
</strike></i>

In [ ]:
#медиана по столбцу floors_total
#floors_total_median = data['floors_total'].median() 
#floors_total_median

In [ ]:
#создание столбца, где номер этажа больше чем медиана floors_total
#floors_to_add = data[(data['floors_total'].isna()) & (data['floor']>=floors_total_median)]['floor']
#print(floors_to_add)
#data.loc[(data['floors_total'].isna()) & (data['floor']>=floors_total_median),'floors_total'] = floors_to_add
#data['floors_total'].isna().sum()
#осталось заполнить

In [ ]:
#data['floors_total'] = data['floors_total'].fillna(floors_total_median)
#data['floors_total'].isna().sum()
#осталось заполнить

**Жилая площадь в квадратных метрах (м²).**

In [ ]:
#living_area - жилая площадь
data['living_area'].isna().sum()


1903 - всего пропусков. Заменим медианным значением.

In [ ]:
print('Медиана living_area')
print(data['living_area'].median())
data['living_area'] = data['living_area'].fillna(data['living_area'].median())
print('Осталось пропусков')
print(data['living_area'].isna().sum())

**Апартаменты или нет.**

In [ ]:
#кол-во пропусков
print(data['is_apartment'].isna().sum())
#что есть в столбце is_apartment 
data.groupby('is_apartment')['is_apartment'].count()

<i><strike>Подавляющая часть столбца не заполнена, а апартаментов совсем мало. Здесь заменять пропуски не нахожу смысла.</strike></i>  
Заполним пустоты как False.

In [ ]:
data['is_apartment'] = data['is_apartment'].fillna(False)
print(data['is_apartment'].isna().sum())


**Площадь кухни в квадратных метрах.**

In [ ]:
#кол-во пропусков
print(data['kitchen_area'].isna().sum())

Заменим пропуски, но только если в столбце studio значение равно false

In [ ]:
data.loc[(data['studio']==False) & (data['kitchen_area'].isna()==True),'kitchen_area'] = data['kitchen_area'].median()
#проверим
data[data['kitchen_area'].isna() == True ]['studio'].unique()
#data['kitchen_area']


Все случаи, где значения остались пустые, это студии.  
  
  
**Число балконов.**

In [ ]:
#кол-во пропусков
print(data['balcony'].isna().sum())
#какие значения есть в столбце
data['balcony'].unique()
#заполним нулями
data['balcony'] = data['balcony'].fillna(0)
#проверим, после замены
print('После замен пропусков')
print(data['balcony'].isna().sum())

**Название населённого пункта.**

In [ ]:
#кол-во пропусков
print(data['locality_name'].isna().sum())

<i><strike>49 строк без названия насленного пункта. Оставляю без изменений.</strike></i>  
Удаляем незаполненные.

In [ ]:
data = data.dropna(subset=['locality_name'])
print(data['locality_name'].isna().sum())

**Расстояние до ближайшего аэропорта в метрах.**

In [ ]:
#кол-во пропусков
print(data['airports_nearest'].isna().sum())

5542 пропуска, оставляю без изменений.  

**Расстояние до центра города.**

In [ ]:
#кол-во пропусков
print(data['city_centers_nearest'].isna().sum())

5519 пропусков, оставляю без изменений.  


**Число парков в радиусе 3 км и расстояние до ближайшего парка (м).**

In [ ]:
#кол-во пропусков
print(data['parks_around3000'].isna().sum())
print(data['parks_nearest'].isna().sum())

Посмотрим на значения стобца parks_around3000

In [ ]:
data['parks_around3000'].unique()

Посмотрим, что в столбце parks_nearest в строках, где в parks_around3000 пропуски и нули.

In [ ]:
print(data[data['parks_around3000'].isna()]['parks_nearest'].unique())
print(data[data['parks_around3000']==0]['parks_nearest'].unique())

В первом случае везде пропуски, во втором пропуски и значения больше 3000 м. Заполняем пропуски в parks_around3000 нулями. Пропуски в parks_nearest не меняем.  



In [ ]:
data['parks_around3000'] = data['parks_around3000'].fillna(0)
#проверим, после замены
print('После замен пропусков')
print(data['parks_around3000'].isna().sum())

**Число водоёмов в радиусе 3 км и расстояние до ближайшего водоёма (м)**

In [ ]:
# по аналогии с парками
# пропуски
print(data['ponds_around3000'].isna().sum())
print(data['ponds_nearest'].isna().sum())

In [ ]:
#уник.значения ponds_around3000
data['ponds_around3000'].unique()

In [ ]:
#что в ponds_nearest, там где в ponds_around3000 ноль или пусто
print(data[data['ponds_around3000'].isna()]['ponds_nearest'].unique())
print(data[data['ponds_around3000']==0]['ponds_nearest'].unique())

Там где ponds_around3000 не заполнено, ponds_nearest не заполнено тоже. Заполняем пропуски в ponds_around3000 нулями. Пропуски в ponds_nearest не меняем.

In [ ]:
data['ponds_around3000'] = data['ponds_around3000'].fillna(0)
#проверим, после замены
print('После замен пропусков')
print(data['ponds_around3000'].isna().sum())

**Cколько дней было размещено объявление.**

In [ ]:
#кол-во пропусков
print(data['days_exposition'].isna().sum())

Не все объявления были сняты, т.к. не все объекты недвижимости проданы. Здесь нет смысла ничего менять.  

**Причины пропусков.**

Основных причин две. Первое - это нежелание людей по какой-либо причине указывать сооответствующие данные, либо же человеку просто не приходит в голову указывать информацию, которая к его квартире не относится, число балконов, например, если в квартире нет балкона. Кроме этого возможны потери данных при загрузке/выгрузке информации из одного источника в другой, это может быть как человеческтй фактор - не указали, что какой-то столбец надо загружать/выгружать, так и технический фактор - информационная система не смогла правильно  обработать столбец и данные не загрузились.



**Изменение типов данных.**

Изменить данные нужно в столбце first_day_exposition, чтобы этот столбец удобнее было использовать в дальнейших вычислениях.

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
data['first_day_exposition'].head()

Кроме этого нужно перевести вещественные числа в целые в столбцах "всего этажей в доме", "число балконов", "число парков в радиусе 3 км", "число водоёмов в радиусе 3 км".

In [ ]:
#до
print(data['floors_total'].dtype, data['balcony'].dtype, data['parks_around3000'].dtype, data['ponds_around3000'].dtype)
data[['floors_total','balcony','parks_around3000','ponds_around3000']]= data[['floors_total','balcony','parks_around3000','ponds_around3000']].astype('int')
#после
print(data['floors_total'].dtype, data['balcony'].dtype, data['parks_around3000'].dtype, data['ponds_around3000'].dtype)

#data.info()

Исходно значение столбца is_apartment было object, но пропуски были заполнены как False. Проверим значение столбца сейчас.

In [ ]:
print(data['is_apartment'].dtype)

В столбце булев тип, изменения не нужны.

**Устранение неявных дубликатов.**  
Неявные дубликаты есть в столбце с названиями населенных пунктов locality_name.

In [ ]:
data['locality_name'] = (
    data['locality_name']
    .str.replace('ё', 'е')
    .replace(['городской поселок ', 'поселок при железнодорожной станции ', 'коттеджный поселок ',
    'поселок станции ', 'деревня ','поселок городского типа ',
    'садовое товарищество ','поселок ', 'садоводческое некоммерческое товарищество ',
    'при железнодорожной станции ', 'село '], '', regex=True)
) 
#вывод уникальных значений для проверки
data['locality_name'].sort_values().unique()

На этапе предобработки данных проверены явные задвоения, в отдельных интересующих столбцах заполнены пропуски, изменен с столбец датами для возможности дальнейших операций с ними. Кроме этого обработан столбец с названиями населенных пунктов, разные варианты записи одного и того же населенного пункта приведены к одинакому виду.

### Посчитайте и добавьте в таблицу новые столбцы


**Цена одного квадратного метра**
square_meter_price

In [ ]:
data['square_meter_price'] = (data['last_price'] / data['total_area']).round(2) 
data['square_meter_price'].head()

**День недели публикации.**

In [ ]:
data['weekday'] = pd.DatetimeIndex(data['first_day_exposition']).weekday
data['weekday'].head()

**Месяц публикации.**

In [ ]:
data['month'] = pd.DatetimeIndex(data['first_day_exposition']).month
data['month'].head()

**Год публикации.**

In [ ]:
data['year'] = pd.DatetimeIndex(data['first_day_exposition']).year
data['year'].head()

**Тип этажа квартиры.**  


In [ ]:
data['floor_type'] = 'другой'
data.loc[data['floor'] == data['floors_total'],'floor_type'] = 'последний'
data.loc[data['floor'] == 1,'floor_type'] = 'первый'
data.groupby('floor_type')['floor_type'].count()

**Расстояние до центра города в километрах.**  


In [ ]:
data['city_сenter_distance'] = (data['city_centers_nearest'] / 1000).round(0)
data['city_сenter_distance'].head()

На этом этапе добавлены столбцы, не содержащиеся в исходных данных, но которые понадобятся для дальнейших действий. 

### Проведите исследовательский анализ данных

Рассмотрим определенные параметры и построим для них диаграммы.  

**Общая площадь.**


In [ ]:
print(data['total_area'].describe())
data['total_area'].hist(bins=30)
#plt.suptitle("suptitle")
#plt.plot('x', 'y')
plt.title("Распределение квартир\n с точки зрения общей площади\n", fontsize=12, fontweight="bold")
plt.xlabel("\nОбщая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")



Максимум 900, но больше 300 уже практически ничего не видно. Построим с параметром range.

In [ ]:
data['total_area'].hist(bins=30, range=(0,300))
plt.title("Распределение квартир\n с точки зрения общей площади.\nВыборка до 300 кв. метров.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nОбщая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Это похоже на распределение Пуассона, пик приходится на 40-50 м2, квартиры по 12 и 900 кв. метров тоже, полагаю, бывают. Здесь ничего не меняю.  

Большинство квартир - это квартиры по 30-50 кв.метров, также достаточно много квартир по 50-70 кв.метров. Квартир больше 70 кв.метров уже существенно меньше - плавное снижение на графике, при этом квартир меньше 30 кв. метров очень мало - график резко обрывается.

**Жилая площадь.**

In [ ]:
print(data['living_area'].describe())
data['living_area'].hist(bins=30)
plt.title("Распределение квартир\n с точки зрения жилой площади.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nЖилая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Максимум 409, но больше 200 уже практически ничего не видно. Минимум 2 неправдоподобен. Отсортируем по возрастанию.

In [ ]:
data.sort_values(by='living_area')['living_area'].head(10)

Меньше 5 кв. метров всё же, думаю, не бывает. Удалим строки меньше 5.


In [ ]:
indexes_to_drop = data.index[data['living_area']<5]
indexes_to_drop
data = data.drop(indexes_to_drop,axis=0)
len(data)

In [ ]:
#data[data['rooms']==1]['living_area'].hist(bins=30,range=(0,100))
data['living_area'].hist(bins=50,range=(0,100))
plt.title("Распределение квартир\n с точки зрения жилой площади.\nВыборка до 100 кв. метров.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nЖилая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")


Это не слишком похоже на распределение Пуассона, т.к. есть два пика, около 18 и около 30. Но возможно такие пики определяются тем, что при увеличении кол-ва жилых комнат, жилая площадь и увеличивается скачкообразно. Построим еще два графика, но с учетом количества комнат.

In [ ]:
data[data['rooms']==1]['living_area'].hist(bins=30,range=(0,100))
data[data['rooms']==2]['living_area'].hist(bins=30,range=(0,100))

plt.title("Распределение квартир\n с точки зрения жилой площади.\nОднокомнатные и двухкомнатные.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nЖилая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Очевидно, что два пика первой гистограммы определяются графиками жилых площадей однокомнатной и двухкомнатной квартир.  

Получилось, что есть два пика, это 18-20 кв. метров, второй пик более выраженный - это 30 кв. метров. После каждого из пиков график снижается достаточно резко, например квартир с жилой площадью 21-27 кв.метров во много раз меньше, чем квартир на любом из пиков. В большую сторону график снижается плавно с 45 кв. метров до 100 кв.метров, в меньшую сторону график резко обрывается на 14 кв.метрах.

**Площадь кухни.**

In [ ]:

print(data['kitchen_area'].describe())
data['kitchen_area'].hist(bins=30, range=(0,50))
#data['kitchen_area']
plt.title("Распределение квартир\n с точки зрения площади кухни.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nПлощадь кухни.", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")



In [ ]:
data[data['kitchen_area'].isna()==False]['kitchen_area'].sort_values().tail(30)

Это похоже на распределение Пуассона, пик около 10, максимум 112, с другой стороны если есть в квартиры в сотни квадратных метров, почему бы там не быть кухням в 112 кв. метров. Не нахожу поводов выбрасывать такие значения только потому, что они слишком большие.

Получилось, что большинство кухонь находятся в диапазоне от 5 до 12 кв.метров. Кухонь площадью от 15 до 25 кв. метров относительно мало. Кухни площадью больше 25 или меньше 5 кв. метров очень редки.

**Цена объекта.**

In [ ]:
print(data['last_price'].describe())
(data['last_price']/1000000).hist(bins=50, range=(0,20))
plt.title("Распределение квартир\n с точки зрения цены.\n\n", fontsize=12, fontweight="bold")
#x_labels = data['last_price']#*1000000
plt.xlabel("\nЦена, млн.руб.", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Без параметра range гистограмма была не информативна, с параметром range это похоже на распределение Пуассона, пик на 3,5-4 млн. рублей.

Получилось, что большинство квартир находятся в ценовом диапазоне от 3 до 5 млн.руб. В обе стороны от этих значений график снижается достаточно плавно, хотя в меньшую сторону и более резко, меньше 400 тыс. рублей предложений нет. В большую сторону совсем мало предложений становится примерно на цене в 20 млн.руб.

In [ ]:
data['last_price'].sort_values()

Квартиру с ценой 12 тыс. рублей удаляю из датафрейма, остальное оставляю.


In [ ]:
indexes_to_drop = data.index[data['last_price']<100000]
indexes_to_drop
data = data.drop(indexes_to_drop,axis=0)
len(data)

**Количество комнат.**

In [ ]:
print(data['rooms'].describe())
data['rooms'].hist(bins=30)
plt.title("Распределение квартир\n с точки зрения количества комнат.\nПредварительный вариант.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nКоличество комнат", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

In [ ]:
data['rooms'].hist(bins=40,range=(0,10))
plt.title("Распределение квартир\n с точки зрения количества комнат.\nДо десяти комнат.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nКоличество комнат", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

19 комнат в очень большой квартире может быть, а ноль комнат быть не может. Даже если это студия, то одна комната все равно есть. Нули заменяю единицей.

In [ ]:
data.loc[(data['rooms']==0),'rooms'] = 1
data['rooms'].hist(bins=20,range=(0,10))
plt.title("Распределение квартир\n с точки зрения количества комнат.\nИтоговый вариант.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nКоличество комнат", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Это тоже похоже на распределение Пуассона.  

Получилось, что самый, почти одинаково популярный вариант - это квартиры с одной или двумя комнатами. Несколько меньше предложений с тремя комнатами. Предложений с четырьмя комнатами уже во много раз меньше. Предложения с пятью и более комнатами очень редки по сравнению с остальными.

**Высота потолков.**

In [ ]:
print(data['ceiling_height'].describe())
data['ceiling_height'].hist(bins=20, range=(0,10))
plt.title("Распределение квартир\n с точки зрения высоты потолков.\nПредварительный вариант.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nВысота потолков", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

In [ ]:
data['ceiling_height'].sort_values().unique()
#посмотрим какие разные значения встречаются 

Видим, что есть целые значения больше 20 - можно предположить, что пропустили запятую. Т.е. превратим, например, 24 в 2,4, но при этом 22,6 и 27,5 менять таким способом не будем, т.к. предполагаю, что здесь может быть другая причина возникновения неадекватного значения.


In [ ]:
data.loc[(data['ceiling_height']==24) | (data['ceiling_height']==25) | (data['ceiling_height']==26) | (data['ceiling_height']==27) | (data['ceiling_height']==32),'ceiling_height'] /= 10
#какого-то более умного способа не нашел
data['ceiling_height'].sort_values().tail(10)

Судя по гистограмме выше значения меньше 2 метров и больше 4,5 метров очень редки. Их заменяю медианным значением.

In [ ]:
data.loc[(data['ceiling_height']<2) | (data['ceiling_height']>4.5),'ceiling_height'] = data['ceiling_height'].median()
data['ceiling_height'].hist(bins= 20,range=(2,4))
plt.title("Распределение квартир\n с точки зрения высоты потолков.\nИтоговый вариант.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nВысота потолков", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

После преобразований, это тоже похоже на распределение Пуассона.  

Получилось, что большинство потолков около 2,7 метра, также много предложений в 2,5 метра, встречаются предложения с потолками в 3 метра, остальные варианты очень редки. 

**Тип этажа квартиры: «первый», «последний» и «другой».**

In [ ]:
print(data['floor_type'].describe())
data['floor_type'].hist()
plt.title("Распределение квартир\n с точки этажа.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nЭтаж", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Здесь три разных значения, как и должно быть, количества параметра "другой" ожидаемо гораздо больше остальных вместе взятых. Ничего не меняю здесь.  

**Общее количество этажей в доме.**

In [ ]:
print(data['floors_total'].describe())
data['floors_total'].hist(bins=30,range=(0,30))
plt.title("Распределение квартир\n с точки количества этажей в доме.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nКоличество этажей", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

In [ ]:
data['floors_total'].sort_values().tail(10)


Гистограмма в общем похожа на распределение Пуассона, ничего выдающегося нет, кроме двух домов в 52 и 60 этажей. Посмотрим в каких городах эти дома.

In [ ]:
data[data['floors_total']>50][['locality_name','floors_total']]

Поскольку эти дома в Кронштадте и Санкт-Петербурге, полагаю здесь тоже можно ничего не менять.

Получилось, что большинство домов - это пяти- и девятиэтажки. В несколько раз меньше домов с 4, 10, 12, 16, 25 этажами, на такие дома тоже встречаются. Остальные варианты более редки.

**Расстояние до центра города в метрах.**

In [ ]:
print(data['city_centers_nearest'].describe())
data['city_centers_nearest'].hist(bins=20)
plt.title("Распределение квартир\n с точки зрения расстояния до центра города.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nДо центра города, м", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Пик приходится на 10-16 км. Подавляющая часть приходится на расстояние до 20 км., это вероятно Санкт-Петербург, максимальное расстояние 65,97 км, что тоже выглядит правдоподобно, как один из населенных пунктов Ленинградской области. 

In [ ]:
data[data['city_centers_nearest']<20000].groupby('locality_name')['locality_name'].count()

Парголово и Шушары граничат с Санкт-Петербургом, остальное собственно Санкт-Петербург.  
Ничего не меняю здесь.  

**Расстояние до ближайшего парка.**

In [ ]:
print(data['parks_nearest'].describe())
data['parks_nearest'].hist(bins=20)
plt.title("Распределение квартир\n с точки зрения расстояния до ближайшего парка.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nДо ближайшего парка, м", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Пик около 500 метров. Похоже на распределение Пуассона. Строк чуть больше 8000, т.е. у большинства домов парков рядом просто нет. Не нахожу, что здесь имеет смысл менять.   

Получилось, что парки есть только у одной трети предложений. Но если парк есть, то расстояние до него не больше километра, остальные варианты очень редки.

**Расстояние до ближайшего аэропорта.**

In [ ]:
print(data['airports_nearest'].describe())
data['airports_nearest'].hist(bins=50, range=(0,60000))
data['airports_nearest'].sort_values().head(5)
plt.title("Распределение квартир\n с точки зрения расстояния до ближайшего аэропорта.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nДо ближайшего аэропорта, м", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Расстояние от 6,5 км до 84 км. Заполнены не все строки, 18 тыс. строк из 23,7 тыс. Видимо по каким-то причинам часть информации получить или загрузить не удалось. Пустых строк слишком много, чтобы удалять их. Явно выбивается строка с 0. Заменим 0 на еще одну пустую строку.

In [ ]:
data.loc[data['airports_nearest']==0,'airports_nearest'] = None
#print(data.loc[21085])
#print(data['airports_nearest'].describe())


Получилось, что с точки зрения расстояния до ближайшего аэропорта значения распределены достаточно равномерно. Большинство это 10-28 км, значений от 28 до 55 км значений в основном меньше, но их тоже много. Явно меньше значений на расстояниях меньше 10 и больше 55 км.

**День недели публикации.**

In [ ]:
data['weekday'].hist(bins=7)
plt.title("Распределение квартир\n с точки зрения дня недели.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nДень недели", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")


Очевидно, что в выходные объявления размещают примерно в два раза реже, чем в будние дни.

**Месяц публикации.**

In [ ]:
data['month'].hist(bins=12,range=(1,12))
data.groupby('month')['month'].count()
plt.title("Распределение квартир\n с точки зрения месяца публикации.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nМесяц публикации", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")

Из гистограммы видно, что количество объявлений больше всего в феврале-апреле, в "праздничные" месяцы (январь, май, декабрь) оно резко падает относительно предыдущих периодов. После майского падения, начиная с июня, количество объявлений постепенно растет, к ноябрю почти достигая значений начала года.

**Насколько быстро продавались квартиры.**

In [ ]:
print(data['days_exposition'].describe())
data['days_exposition'].hist(bins=20)
plt.title("Насколько быстро продавались квартиры.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nДни", fontsize=12, fontweight="bold")
plt.ylabel("Количество квартир\n",  fontsize=12, fontweight="bold")


In [ ]:

print('Медианное значение: ' + str(data['days_exposition'].median()))
print('Среднее значение: ' + str(data['days_exposition'].mean().round(2)))
data.boxplot('days_exposition') 
plt.ylim(-50, 800)
#plt.xlim(0, 200) 
plt.title("Диаграмма размаха\nдней до продажи\n", fontsize=12, fontweight="bold")
plt.xlabel("\n", fontsize=12, fontweight="bold")
plt.ylabel("Дни\n",  fontsize=12, fontweight="bold")

Продажа обычно занимает 95 дней. Быстрыми продажами могут считаться продажи до 45 дней (попадающие в первый квартиль). Долгими продажи могут считаться продажи больше 232 дней (3 квартиль и дальше), и особенно долгими продажи больше 512, т.е. продажи из зоны выбросов.  

**Какие факторы влияют на цену объекта.**  
Рассмотрим зависит ли цена от следующих факторов.  
 
**Общая площадь.**


In [ ]:
#data.sort_values('last_price').plot(x='total_area', y='last_price',kind='scatter')
data.sort_values('last_price').plot(x='total_area', y='last_price',kind='scatter')
plt.title("Зависимость цены от общей площади.\nПредварительный вариант.\n", fontsize=12, fontweight="bold")
plt.xlabel("\nОбщая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")


По такому графику немного можно сказать. Построим используя range.

In [ ]:
data.sort_values('last_price').plot(x='total_area', y='last_price',kind='scatter',alpha=0.15,xlim=(0,400), ylim=(0,50000000))
plt.title("Зависимость цены от общей площади.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nОбщая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")


По такому графику уже можно сделать вывод - чем больше площадь, тем больше цена. <i><strike>зависимость прямая.  </strike></i>

Найдем корреляцию между ценой и общей площадью.

In [ ]:
print(data['last_price'].corr(data['total_area']))

Зависимость есть, не прямая, но существенная.

**Жилая площадь.**

In [ ]:
data.sort_values('last_price').plot(x='living_area', y='last_price',kind='scatter',alpha=0.15,xlim=(0,400), ylim=(0,50000000))
plt.title("Зависимость цены от жилой площади.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nЖилая площадь", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")


Для жилой площади можно такой же вывод, зависимость прямая, и она больше чем у общей площади.  
Кроме этого здесь бросается в глаза вертикальная "линия" около 30 кв.метров, а также не такая явная, но тоже заметная горизонтальная "линия" около 40 млн. руб. И то, и другое выглядит неправдоподобно, возможно это еще одна ошибка в исходных данных.

In [ ]:
data.sort_values('last_price').plot(x='living_area', y='last_price',kind='scatter',alpha=0.15,xlim=(29,31), ylim=(0,50000000))
plt.title("Неправдоподобная прямая линия \nна 30 кв.метрах.\n", fontsize=12, fontweight="bold")
#plt.xlabel("\nОбщая площадь", fontsize=12, fontweight="bold")
#plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")


Найдем корреляцию между ценой и жилой площадью.

In [ ]:
print(data['last_price'].corr(data['living_area']))
#print(data['living_area'].corr(data['last_price']))

Зависимость есть, она достаточно велика, но оказалось, что корреляция между ценой и общей площадью выше.

In [ ]:
data.sort_values('last_price').plot(x='living_area', y='last_price',kind='scatter',alpha=0.5,xlim=(0,400), ylim=(35000000,45000000))
plt.title("Неправдоподобная прямая линия \nна 40 млн.руб.\n", fontsize=12, fontweight="bold")

**Площадь кухни.**

In [ ]:
data.sort_values('last_price').plot(x='kitchen_area', y='last_price',kind='scatter',alpha=0.15,xlim=(0,50), ylim=(0,50000000))
plt.title("Зависимость цены от площади кухни.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nПлощадь кухни", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

In [ ]:
#data.pivot_table(index='kitchen_area', values='last_price', aggfunc=sum)
data.groupby('kitchen_area')['last_price'].count().sort_values().tail(10)

In [ ]:
data[data['kitchen_area']<=20].sort_values('last_price').plot(x='kitchen_area', y='last_price',kind='scatter',alpha=0.15,xlim=(0,21), ylim=(0,50000000))
plt.title("Часто встречающиеся значения \nплощади кухни.\n", fontsize=12, fontweight="bold")

В целом тоже есть прямая зависимость, чем больше площадь кухни, тем больше цена. Одновременно с этим для площади кухни характерен набор верикальных "линий", когда для кухня одного размера может быть в квартирах очень разной цены. Но в отличии от графика жилой площади здесь, полагаю, это не выглядит явной ошибкой.

Найдем корреляцию между ценой и площадью кухни.

In [ ]:
print(data['last_price'].corr(data['kitchen_area']))

Зависимость есть, но она еще меньше, чем от жилой площади. 

**Количество комнат.**

In [ ]:
rooms_price_dependance = data.pivot_table(index='rooms', values='last_price', aggfunc=['mean','median','count'])
#rooms_price_dependance
rooms_price_dependance.columns = ('mean','median','count')
rooms_price_dependance.plot(y='median', xlim=(0,8), ylim=(0,30000000),grid=True)
print(rooms_price_dependance)
plt.title("Зависимость цены от количества комнат.\n\n", fontsize=12, fontweight="bold")
plt.xlabel("\nКоличество комнат", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

За основной показатель возьмем медиану, больше 8 комнат брать не будем, таких случаев слишком мало. Очевидна прямая зависимость между количеством комнат и ценой, если количество комнат 7 или меньше. Начиная с 8 комнат, случаев мало, каждый индивидуален, поэтому говорить о зависимости не приходится.

Найдем корреляцию между ценой и количеством комнат.

In [ ]:
print(data['last_price'].corr(data['rooms']))

Зависимость есть, она невелика, и ещё меньше, чем у предыдущих показателей.

**Этаж (первый, последний, другой).**

In [ ]:
#floor_type
floortype_price_dependance = data.pivot_table(index='floor_type', values='last_price', aggfunc=['mean','median','count'])
floortype_price_dependance.columns = ('mean','median','count')
floortype_price_dependance.plot(y='median', xlim=(0,2), ylim=(3500000,5500000))
print(floortype_price_dependance)
plt.title("Зависимость цены от этажа\n", fontsize=12, fontweight="bold")
plt.xlabel("\nЭтаж", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

За основной показатель возьмем медиану. Самые дешевые квартиры на первом этаже, дороже на последнем, наиболее дорогие на "других", т.е. не на крайних этажах.

**День, месяц и год.**

In [ ]:
data.pivot_table(index='weekday', values='last_price',aggfunc=['mean','median','count']).plot(y='median',grid=True)
plt.title("Зависимость цены от дня недели\n", fontsize=12, fontweight="bold")
plt.xlabel("\nДень недели", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

График цены с точки зрения дня недели - не прямая линия, есть некоторые колебания, но нельзя сказать что они существенны.

Найдем корреляцию между ценой и днем недели.

In [ ]:
print(data['last_price'].corr(data['weekday']))

Зависимость практически отсутствует.

In [ ]:
data.pivot_table(index='month', values='last_price',aggfunc=['mean','median','count']).plot(y='median',grid=True)
plt.title("Зависимость цены от месяца\n", fontsize=12, fontweight="bold")
plt.xlabel("\nМесяц", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

С точки месяца, в течение года есть колебания, выделяются - высокие цены в апреле, низкие цены - в мае и особенно в июне.

Найдем корреляцию между ценой и месяцем.

In [ ]:
print(data['last_price'].corr(data['month']))

Получается, что в целом зависимость тоже практически отсутствует.

In [ ]:
data.pivot_table(index='year', values='last_price',aggfunc=['mean','median','count'])

In [ ]:
data.pivot_table(index='year', values='last_price',aggfunc=['mean','median','count']).plot(y='median',grid=True)
plt.title("Зависимость цены от года\n", fontsize=12, fontweight="bold")
plt.xlabel("\nГод", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

In [ ]:
data.pivot_table(index='year', values='last_price',aggfunc=['mean','median','count']).plot(y='count',grid=True)
plt.title("Зависимость количества объявлений \nот года\n", fontsize=12, fontweight="bold")
plt.xlabel("\nГод", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")


С точки зрения года, если смотреть на два графика, цены и кол-ва объявлений, то окажется, что цена обратно пропорциональна кол-ву объявлений. Чем больше объявлений, тем ниже цена, а кол-во объявлений от года к году сильно отличается. Поэтому здесь однозначных выводов сделать нельзя, сначала надо разобраться, почему такое разное количество объявлений.  

Найдем корреляцию между ценой и годом.

In [ ]:
print(data['last_price'].corr(data['year']))

Здесь зависимость тоже близка к нулю.

**Средняя цена одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений.**

In [ ]:
#data.head()

In [ ]:
max_offers = data.pivot_table(index='locality_name',values='last_price', aggfunc='count').sort_values(by='last_price', ascending=False).head(10)
#ten_popular_locs = data.query('locality_name in @max_offers.index').pivot_table(index='locality_name',values='square_meter_price', aggfunc='mean').sort_values(by='square_meter_price', ascending=False).style.format({'square_meter_price': '{:.0f}'})
ten_popular_locs =(
    data.query('locality_name in @max_offers.index')
    .pivot_table(index='locality_name',values='square_meter_price', aggfunc='mean')
    .sort_values(by='square_meter_price', ascending=False)
    #.style.format({'square_meter_price': '{:.0f}'})
)
ten_popular_locs

In [ ]:
print('Населённый пункт с самой высокой стоимостью квадратного метра: ' + str(ten_popular_locs.index[0]))
print('Населённый пункт с самой низкой стоимостью квадратного метра: ' + str(ten_popular_locs.index[9]))
# 0 и 9, т.к. сами выбрали, что должно быть 10 строк

In [ ]:
plt.plot(ten_popular_locs['square_meter_price'])
plt.xticks(rotation=90)
plt.title("Средняя цена кв.метра ", fontsize=12, fontweight="bold")
plt.grid() 

**Зависимость цены жилья от расстояния от центра города (на примере Санкт-Петербурга).**

In [ ]:
(
    data[data['locality_name']=='Санкт-Петербург'].
    pivot_table(index='city_сenter_distance',values='square_meter_price',aggfunc='mean')
    .plot(grid=True)
)
plt.title("Зависимость цены от расстояния\nдо центра города\n", fontsize=12, fontweight="bold")
plt.xlabel("\nКилометры", fontsize=12, fontweight="bold")
plt.ylabel("Цена\n",  fontsize=12, fontweight="bold")

Цена кв. метра по мере удаления от центра города в целом падает, особенно резкое падение на первых трех километрах. Далее более плавное падение с отдельными всплесками, обусловленными какими-то другими факторами. 


На этом этапе более подробно изучены некоторые параметры исходных данных, выбивающиеся неадекватные значения удалены. Рассмотрено как быстро продавались квартиры. Рассмотрена зависимость цены от общей и жилой площади, площади кухни, количества комнат, этажа и даты размещения. Рассчитана средняя цена квадратного метра в 10 населенных пунктах с наибольшим количеством объявлений. Также рассмотрена зависимость средней цены квадратного метра от расстояния от центра Санкт-Петербурга.

### Общий вывод

Целью задачи было установить параметры влияющие на цену объекта недвижимости. По итогу работы можно сделать следующие выводы:   
    - с точки зрения площади, цена напрямую зависит от жилой площади, в меньшей степени от общей площади, в еще меньшей степени от площади кухни;  
    - с точки зрения количества комнат, цена зависит напрямую от количества комнат, но до определенной степени, при большом количестве комнат явной зависимости нет;  
    - с точки зрения этажа, не первый и не последний этаж самые дорогие, дешевле жилье на последнем этаже, самое дешевое на первом этаже;  
    - с точки зрения даты размещения, выделяются самые высокие цены в апреле, низкие цены в мае и особенно в июне, влияние дня недели несущественно;  
    - наибольшая средняя цена квадратного метра в Санкт-Петербурге, Пушкине и Кудрово;  
    - если брать 10 наиболее популярных с точки зрения количества объявлений населенных пунктов, то средняя цена в самом популярном населенном пункте, Санкт-Петербурге, почти в два раза отличается от цены последнего по популярности, Выборга;  
    - цена жилья в Санкт-Петербурге явно зависит от расстояния от центра города, особенно это проявляется на первых трех километрах, далее зависимость в целом тоже сохраняется, но возможно и влияние других факторов.
    

**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл
- [x]  файлы изучены (выведены первые строки, метод `info()`, гистограммы и т.д.)
- [x]  определены пропущенные значения
- [x]  заполнены пропущенные значения там, где это возможно
- [x]  есть пояснение, какие пропущенные значения обнаружены
- [x]  изменены типы данных
- [x]  есть пояснение, в каких столбцах изменены типы и почему
- [x]  устранены неявные дубликаты в названиях населённых пунктов
- [x]  устранены редкие и выбивающиеся значения (аномалии) во всех столбцах
- [x]  посчитано и добавлено в таблицу: цена одного квадратного метра
- [x]  посчитано и добавлено в таблицу: день публикации объявления (0 - понедельник, 1 - вторник и т.д.)
- [x]  посчитано и добавлено в таблицу: месяц публикации объявления
- [x]  посчитано и добавлено в таблицу: год публикации объявления
- [x]  посчитано и добавлено в таблицу: тип этажа квартиры (значения — «первый», «последний», «другой»)
- [x]  посчитано и добавлено в таблицу: расстояние в км до центра города
- [x]  изучены и описаны следующие параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - этаж квартиры;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего аэропорта;
        - расстояние до ближайшего парка;
        - день и месяц публикации объявления
- [x]  построены гистограммы для каждого параметра
- [x]  выполнено задание: "Изучите, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней «висело» каждое объявление.
    - Постройте гистограмму.
    - Посчитайте среднее и медиану.
    - В ячейке типа markdown опишите, сколько обычно занимает продажа. Какие продажи можно считать быстрыми, а какие — необычно долгими?"
- [x]  выполнено задание: "Какие факторы больше всего влияют на общую (полную) стоимость объекта? Постройте графики, которые покажут зависимость цены от указанных ниже параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы."
        - общей площади;
        - жилой площади;
        - площади кухни;
        - количество комнат;
        - типа этажа, на котором расположена квартира (первый, последний, другой);
        - даты размещения (день недели, месяц, год);
- [x]  выполнено задание: "Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра. Эти данные можно найти по имени в столбце `locality_name`."
- [x]  выполнено задание: "Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите среднюю цену каждого километра. Опишите, как стоимость объектов зависит от расстояния до центра города."
- [x]  в каждом этапе есть промежуточные выводы
- [x]  есть общий вывод